# v2.1 exploration trying to make it work better

In [ ]:
# Environment
import os
import os.path as osp
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
import pandas as pd
from utils import print_dict_summary
from data_funcs import rmse, process_train_dict
from moisture_rnn import RNNParams, RNNData, RNN, RNN_LSTM
from moisture_rnn_pkl import pkl2train
from tensorflow.keras.callbacks import Callback
from utils import hash2
import copy
import logging
import pickle
from utils import logging_setup, read_yml, read_pkl, hash_ndarray, hash_weights
import yaml
import copy

In [ ]:
logging_setup()

## Tests

In [ ]:
from itertools import islice
from data_funcs import combine_nested
from utils import Dict
params_data = read_yml("params_data.yaml")
params = read_yml("params.yaml", subkey="rnn")
params = RNNParams(params)

In [ ]:
params_data['hours']

In [ ]:
train = process_train_dict(["data/fmda_nw_202401-05_f05.pkl"], params_data = params_data, verbose=True)

In [ ]:
train = {k: train[k] for k in islice(train, 100)}
dd = combine_nested(train)
dd = Dict(dd)

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNNData

In [ ]:
rnn_dat = RNNData(dd, scaler="standard", 
                  features_list = ['Ed', 'Ew', 'rain', 'elev', 'lon', 'lat'])
rnn_dat.train_test_split(   
    time_fracs = [.8, .1, .1],
    space_fracs = [.8, .1, .1]
)
params.update({'batch_size': 32})
rnn_dat.batch_reshape(
    timesteps = params['timesteps'], 
    batch_size = params['batch_size'],
    start_times = np.zeros(len(rnn_dat.case)).astype(int),
    verbose=False
)

In [ ]:
params.update({'batch_schedule_type': 'exp', 'bmin': 20, 'bmax': rnn_dat.hours,
               'loc_batch_reset': rnn_dat.n_seqs, 
               'epochs': 100, 'learning_rate': 0.001,
               'activation': ['tanh', 'tanh'],
               'recurrent_layers': 2, 'recurrent_units': 20, 'dense_layers': 1, 'dense_units': 20,
              'features_list': rnn_dat.features_list})

In [ ]:
reproducibility.set_seed(123)
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
errs.mean()

## LSTM

TODO: FIX BELOW

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
params = read_yml("params.yaml", subkey="lstm")
params = RNNParams(params)

In [ ]:
from moisture_rnn import ResetStatesCallback, EarlyStoppingCallback
params.update({'epochs': 50, 'learning_rate': 0.00005, 'verbose_fit': True, 'rnn_layers': 2, 'rnn_units': 20, 'dense_layers': 1, 'dense_units': 10,
              'activation': ['relu', 'relu'], 'features_list': rnn_dat.features_list,
              'batch_schedule_type':'exp', 'bmin': 10, 'bmax':rnn_dat.hours})
reproducibility.set_seed(123)
lstm = RNN_LSTM(params)

history = lstm.model_train.fit(rnn_dat.X_train, rnn_dat.y_train, 
                    batch_size = params['batch_size'], epochs=params['epochs'], 
                    callbacks = [ResetStatesCallback(params),
                                EarlyStoppingCallback(patience = 15)],
                   validation_data = (rnn_dat.X_val, rnn_dat.y_val))
              